# Settings

In [1]:
%env TF_KERAS = 1
import os
sep_local = os.path.sep

import sys
sys.path.append('..'+sep_local+'..')
print(sep_local)

env: TF_KERAS=1
\


In [2]:
import tensorflow as tf
print(tf.__version__)

2.1.0


# Dataset loading

In [3]:
dataset_name='pokemon'

In [4]:
IMG_DIR = 'C:\\Users\\Khalid\\Documents\projects\\pokemon\DS06\\'
VAL_PCT = 10
VAL_FORMAT = 'png'

In [5]:
from training.generators.file_image_generator import create_image_lists, get_generators

Using TensorFlow backend.


In [6]:
imgs_list = create_image_lists(
    image_dir=IMG_DIR, 
    validation_pct=VAL_PCT, 
    valid_imgae_formats=VAL_FORMAT
)

  DEBUG    | Looking for images in 'all'


  INFO     | 809 file found


In [7]:
inputs_shape= IMG_SIZE=(200, 200, 3)
BATCH_SIZE = 32
latent_dim = 32
intermediate_dim = 32

In [8]:
train_gen, test_gen = get_generators(
    images_list=imgs_list, 
    image_dir=IMG_DIR, 
    image_size=IMG_SIZE, 
    batch_size=BATCH_SIZE, 
    class_mode=None
)

  INFO     | Found 738 training files


  INFO     | Found 71 validation files


In [9]:
import tensorflow as tf

In [10]:
train_ds = tf.data.Dataset.from_generator(
    lambda: train_gen, 
    output_types=tf.float32 ,
    output_shapes=tf.TensorShape((BATCH_SIZE, ) + IMG_SIZE)
)

test_ds = tf.data.Dataset.from_generator(
    lambda: test_gen, 
    output_types=tf.float32 ,
    output_shapes=tf.TensorShape((BATCH_SIZE, ) + IMG_SIZE)
)

In [11]:
_instance_scale=1.0
for data in train_ds:
    _instance_scale = float(data[0].numpy().max())
    break

In [12]:
_instance_scale

1.0

In [13]:
import numpy as np
from collections.abc import Iterable

In [14]:
if isinstance(inputs_shape, Iterable):
    _outputs_shape = np.prod(inputs_shape)

In [15]:
_outputs_shape

120000

# Model's Layers definition

In [16]:
enc_lays = [tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units=latent_dim)]

dec_lays = [tf.keras.layers.Dense(units=latent_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=_outputs_shape),
            tf.keras.layers.Reshape(inputs_shape)]

# Model definition

In [17]:
model_name = dataset_name+'AEpsnr'
recoding_dir='..'+sep_local+'..'+sep_local+'recoding'+sep_local+model_name

In [18]:
from training.traditional.autoencoders.autoencoder import autoencoder as AE

In [19]:
inputs_shape=IMG_SIZE

In [20]:
variables_params = \
[
    {
        'name': 'inference', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latent_dim,
        'layers': enc_lays#enc_lays2#
    }

    ,
    
        {
        'name': 'generative', 
        'inputs_shape':latent_dim,
        'outputs_shape':inputs_shape,
        'layers':dec_lays#dec_lays2#
    }
]

In [21]:
from utils.data_and_files.file_utils import create_if_not_exist

In [22]:
_restore = os.path.join(recoding_dir, 'var_save_dir')

In [23]:
create_if_not_exist(_restore)
_restore

'..\\..\\recoding\\pokemonAEpsnr\\var_save_dir'

In [24]:
#to restore trained model, set filepath=_restore

In [25]:
ae = AE( 
    name=model_name,
    inputs_shape=inputs_shape,
    outputs_shape=inputs_shape,
    latent_dim=latent_dim,
    batch_size=BATCH_SIZE,
    variables_params=variables_params, 
    filepath=None
    )

Model: "inference"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 200, 200, 3)]     0         
_________________________________________________________________
dense (Dense)                (None, 200, 200, 32)      128       
_________________________________________________________________
dense_1 (Dense)              (None, 200, 200, 32)      1056      
_________________________________________________________________
flatten (Flatten)            (None, 1280000)           0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                40960032  
_________________________________________________________________
batch_normalization (BatchNo (None, 32)                128       
_________________________________________________________________
dropout (Dropout)            (None, 32)                0 

  WARNING  | None


Model: "generative"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
generative_inputs (InputLaye [(None, 32)]              0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_4 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_5 (Dense)              (None, 120000)            3960000   
_________________________________________________________________
reshape (Reshape)            (None, 200, 200, 3)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 200, 200, 3)       12        
_________________________________________________________________
dropout_1 (Dropout)          (None, 200, 200, 3)       0

  WARNING  | None


In [26]:
from evaluation.quantitive_metrics.psnr import psnr
#from evaluation.quantitive_metrics.ssmi import ssmi
#from evaluation.quantitive_metrics.sharp_diff import sharp_diff


In [27]:
ae.compile(loss={'x_logits': psnr})

Model: "pokemonAEpsnr"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 200, 200, 3)]     0         
_________________________________________________________________
inference (Model)            (None, 32)                40961344  
_________________________________________________________________
generative (Model)           (None, 200, 200, 3)       3962124   
_________________________________________________________________
tf_op_layer_x_logits (Tensor [(None, 200, 200, 3)]     0         
Total params: 44,923,468
Trainable params: 44,923,398
Non-trainable params: 70
_________________________________________________________________
None


# Callbacks

In [28]:
from training.callbacks.progress_bar import NotebookPrograssBar
from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver

In [29]:
progbar = NotebookPrograssBar(leave_outer=False)

In [30]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    min_delta=1e-12, 
    patience=5, 
    verbose=1, 
    restore_best_weights=True
)

In [31]:
ms = ModelSaver(filepath=_restore,save_freq=5)

In [32]:
csv_dir = os.path.join(recoding_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, ae.name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
csv_dir

'..\\..\\recoding\\pokemonAEpsnr\\csv_dir\\pokemonAEpsnr.csv'

In [33]:
image_gen_dir = os.path.join(recoding_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)

In [34]:
sg = SampleGeneration(latent_shape=latent_dim, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=False)

In [35]:
import numpy as np

# Model Training

In [ ]:
ae.fit(
    x=train_ds,
    input_kw=None,
    steps_per_epoch=100,
    epochs=int(1e6), 
    verbose=0,
    callbacks=[progbar, es, ms, csv_log, sg],
    workers=-1,
    use_multiprocessing=True,
    validation_data=test_ds,
    validation_steps=100
)

Train for 50 steps, validate for 50 steps


Epoch 1/50
50/50 [==============================] - ETA: 3:12 - loss: 7.3587 - psnr: 7.3091 - ssmi: 0.0295 - sharp_diff: 6.96 - ETA: 1:40 - loss: 7.3254 - psnr: 7.2721 - ssmi: 0.0291 - sharp_diff: 6.88 - ETA: 1:09 - loss: 7.2966 - psnr: 7.2399 - ssmi: 0.0287 - sharp_diff: 6.83 - ETA: 53s - loss: 7.2514 - psnr: 7.1888 - ssmi: 0.0271 - sharp_diff: 6.6855 - ETA: 44s - loss: 7.2270 - psnr: 7.1598 - ssmi: 0.0272 - sharp_diff: 6.633 - ETA: 37s - loss: 7.2131 - psnr: 7.1413 - ssmi: 0.0266 - sharp_diff: 6.537 - ETA: 33s - loss: 7.1888 - psnr: 7.1119 - ssmi: 0.0251 - sharp_diff: 6.400 - ETA: 29s - loss: 7.1868 - psnr: 7.1056 - ssmi: 0.0245 - sharp_diff: 6.314 - ETA: 26s - loss: 7.1762 - psnr: 7.0921 - ssmi: 0.0237 - sharp_diff: 6.228 - ETA: 24s - loss: 7.1660 - psnr: 7.0790 - ssmi: 0.0232 - sharp_diff: 6.163 - ETA: 22s - loss: 7.1495 - psnr: 7.0602 - ssmi: 0.0225 - sharp_diff: 6.090 - ETA: 21s - loss: 7.1233 - psnr: 7.0315 - ssmi: 0.0218 - sharp_diff: 6.017 - ETA: 19s - loss: 7.1015 - psnr: 7.0

Epoch 2/50
50/50 [==============================] - ETA: 13s - loss: 5.7323 - psnr: 5.5118 - ssmi: 0.0137 - sharp_diff: 5.564 - ETA: 12s - loss: 5.6811 - psnr: 5.4618 - ssmi: 0.0133 - sharp_diff: 5.565 - ETA: 12s - loss: 5.6982 - psnr: 5.4777 - ssmi: 0.0135 - sharp_diff: 5.602 - ETA: 11s - loss: 5.6940 - psnr: 5.4744 - ssmi: 0.0139 - sharp_diff: 5.638 - ETA: 11s - loss: 5.6917 - psnr: 5.4712 - ssmi: 0.0134 - sharp_diff: 5.635 - ETA: 11s - loss: 5.6715 - psnr: 5.4503 - ssmi: 0.0134 - sharp_diff: 5.646 - ETA: 10s - loss: 5.6609 - psnr: 5.4393 - ssmi: 0.0134 - sharp_diff: 5.677 - ETA: 10s - loss: 5.6537 - psnr: 5.4322 - ssmi: 0.0133 - sharp_diff: 5.694 - ETA: 10s - loss: 5.6649 - psnr: 5.4434 - ssmi: 0.0135 - sharp_diff: 5.712 - ETA: 10s - loss: 5.6621 - psnr: 5.4400 - ssmi: 0.0133 - sharp_diff: 5.703 - ETA: 9s - loss: 5.6484 - psnr: 5.4253 - ssmi: 0.0132 - sharp_diff: 5.707 - ETA: 9s - loss: 5.6400 - psnr: 5.4165 - ssmi: 0.0132 - sharp_diff: 5.70 - ETA: 9s - loss: 5.6267 - psnr: 5.4028 -

Epoch 3/50
50/50 [==============================] - ETA: 12s - loss: 5.2801 - psnr: 5.0287 - ssmi: 0.0115 - sharp_diff: 6.055 - ETA: 12s - loss: 5.3309 - psnr: 5.0790 - ssmi: 0.0119 - sharp_diff: 6.150 - ETA: 11s - loss: 5.3521 - psnr: 5.1009 - ssmi: 0.0130 - sharp_diff: 6.175 - ETA: 11s - loss: 5.3574 - psnr: 5.1052 - ssmi: 0.0129 - sharp_diff: 6.160 - ETA: 11s - loss: 5.3471 - psnr: 5.0952 - ssmi: 0.0127 - sharp_diff: 6.157 - ETA: 10s - loss: 5.3414 - psnr: 5.0893 - ssmi: 0.0124 - sharp_diff: 6.165 - ETA: 10s - loss: 5.3578 - psnr: 5.1057 - ssmi: 0.0126 - sharp_diff: 6.166 - ETA: 10s - loss: 5.3709 - psnr: 5.1186 - ssmi: 0.0128 - sharp_diff: 6.169 - ETA: 10s - loss: 5.3597 - psnr: 5.1072 - ssmi: 0.0131 - sharp_diff: 6.173 - ETA: 9s - loss: 5.3438 - psnr: 5.0912 - ssmi: 0.0128 - sharp_diff: 6.178 - ETA: 9s - loss: 5.3387 - psnr: 5.0861 - ssmi: 0.0128 - sharp_diff: 6.17 - ETA: 9s - loss: 5.3338 - psnr: 5.0810 - ssmi: 0.0128 - sharp_diff: 6.18 - ETA: 9s - loss: 5.3249 - psnr: 5.0719 - s

Epoch 4/50
50/50 [==============================] - ETA: 12s - loss: 5.2548 - psnr: 4.9885 - ssmi: 0.0126 - sharp_diff: 6.184 - ETA: 12s - loss: 5.2019 - psnr: 4.9356 - ssmi: 0.0123 - sharp_diff: 6.196 - ETA: 11s - loss: 5.2188 - psnr: 4.9532 - ssmi: 0.0127 - sharp_diff: 6.239 - ETA: 11s - loss: 5.2516 - psnr: 4.9862 - ssmi: 0.0129 - sharp_diff: 6.250 - ETA: 11s - loss: 5.2569 - psnr: 4.9912 - ssmi: 0.0130 - sharp_diff: 6.250 - ETA: 11s - loss: 5.2611 - psnr: 4.9951 - ssmi: 0.0130 - sharp_diff: 6.262 - ETA: 10s - loss: 5.2703 - psnr: 5.0045 - ssmi: 0.0133 - sharp_diff: 6.272 - ETA: 10s - loss: 5.2561 - psnr: 4.9902 - ssmi: 0.0132 - sharp_diff: 6.266 - ETA: 10s - loss: 5.2539 - psnr: 4.9878 - ssmi: 0.0131 - sharp_diff: 6.262 - ETA: 10s - loss: 5.2642 - psnr: 4.9981 - ssmi: 0.0132 - sharp_diff: 6.273 - ETA: 9s - loss: 5.2676 - psnr: 5.0014 - ssmi: 0.0132 - sharp_diff: 6.276 - ETA: 9s - loss: 5.2515 - psnr: 4.9853 - ssmi: 0.0132 - sharp_diff: 6.27 - ETA: 9s - loss: 5.2596 - psnr: 4.9934 -

...

Epoch 18/50
50/50 [==============================] - ETA: 12s - loss: 4.4539 - psnr: 4.0248 - ssmi: 0.0085 - sharp_diff: 5.696 - ETA: 12s - loss: 4.4263 - psnr: 3.9966 - ssmi: 0.0072 - sharp_diff: 5.669 - ETA: 11s - loss: 4.4201 - psnr: 3.9906 - ssmi: 0.0077 - sharp_diff: 5.694 - ETA: 11s - loss: 4.3973 - psnr: 3.9675 - ssmi: 0.0077 - sharp_diff: 5.672 - ETA: 11s - loss: 4.4310 - psnr: 4.0010 - ssmi: 0.0080 - sharp_diff: 5.674 - ETA: 11s - loss: 4.4178 - psnr: 3.9877 - ssmi: 0.0077 - sharp_diff: 5.671 - ETA: 10s - loss: 4.4070 - psnr: 3.9767 - ssmi: 0.0075 - sharp_diff: 5.668 - ETA: 10s - loss: 4.4068 - psnr: 3.9763 - ssmi: 0.0074 - sharp_diff: 5.669 - ETA: 10s - loss: 4.4015 - psnr: 3.9707 - ssmi: 0.0074 - sharp_diff: 5.662 - ETA: 10s - loss: 4.4004 - psnr: 3.9696 - ssmi: 0.0073 - sharp_diff: 5.665 - ETA: 9s - loss: 4.4025 - psnr: 3.9714 - ssmi: 0.0073 - sharp_diff: 5.660 - ETA: 9s - loss: 4.3948 - psnr: 3.9636 - ssmi: 0.0073 - sharp_diff: 5.65 - ETA: 9s - loss: 4.4352 - psnr: 4.0041 

Epoch 19/50
50/50 [==============================] - ETA: 12s - loss: 4.4264 - psnr: 3.9841 - ssmi: 0.0079 - sharp_diff: 5.662 - ETA: 12s - loss: 4.3215 - psnr: 3.8792 - ssmi: 0.0072 - sharp_diff: 5.651 - ETA: 11s - loss: 4.3332 - psnr: 3.8901 - ssmi: 0.0068 - sharp_diff: 5.632 - ETA: 11s - loss: 4.3410 - psnr: 3.8976 - ssmi: 0.0067 - sharp_diff: 5.627 - ETA: 11s - loss: 4.3316 - psnr: 3.8882 - ssmi: 0.0065 - sharp_diff: 5.626 - ETA: 11s - loss: 4.3150 - psnr: 3.8711 - ssmi: 0.0067 - sharp_diff: 5.614 - ETA: 10s - loss: 4.3302 - psnr: 3.8861 - ssmi: 0.0068 - sharp_diff: 5.617 - ETA: 10s - loss: 4.3436 - psnr: 3.8995 - ssmi: 0.0069 - sharp_diff: 5.622 - ETA: 10s - loss: 4.3340 - psnr: 3.8898 - ssmi: 0.0069 - sharp_diff: 5.626 - ETA: 10s - loss: 4.3297 - psnr: 3.8855 - ssmi: 0.0069 - sharp_diff: 5.629 - ETA: 9s - loss: 4.3471 - psnr: 3.9031 - ssmi: 0.0073 - sharp_diff: 5.634 - ETA: 9s - loss: 4.3465 - psnr: 3.9022 - ssmi: 0.0072 - sharp_diff: 5.63 - ETA: 9s - loss: 4.3456 - psnr: 3.9010 

Epoch 20/50
50/50 [==============================] - ETA: 12s - loss: 4.2751 - psnr: 3.8182 - ssmi: 0.0074 - sharp_diff: 5.554 - ETA: 12s - loss: 4.2813 - psnr: 3.8243 - ssmi: 0.0068 - sharp_diff: 5.566 - ETA: 11s - loss: 4.2659 - psnr: 3.8089 - ssmi: 0.0069 - sharp_diff: 5.574 - ETA: 11s - loss: 4.2846 - psnr: 3.8277 - ssmi: 0.0072 - sharp_diff: 5.587 - ETA: 11s - loss: 4.2842 - psnr: 3.8271 - ssmi: 0.0072 - sharp_diff: 5.583 - ETA: 11s - loss: 4.2986 - psnr: 3.8412 - ssmi: 0.0072 - sharp_diff: 5.584 - ETA: 10s - loss: 4.3043 - psnr: 3.8469 - ssmi: 0.0072 - sharp_diff: 5.589 - ETA: 10s - loss: 4.3088 - psnr: 3.8513 - ssmi: 0.0073 - sharp_diff: 5.588 - ETA: 10s - loss: 4.3257 - psnr: 3.8686 - ssmi: 0.0072 - sharp_diff: 5.597 - ETA: 10s - loss: 4.3287 - psnr: 3.8714 - ssmi: 0.0072 - sharp_diff: 5.598 - ETA: 9s - loss: 4.3196 - psnr: 3.8621 - ssmi: 0.0071 - sharp_diff: 5.598 - ETA: 9s - loss: 4.3142 - psnr: 3.8565 - ssmi: 0.0071 - sharp_diff: 5.59 - ETA: 9s - loss: 4.3149 - psnr: 3.8570 

Epoch 21/50
50/50 [==============================] - 19s 389ms/step - loss: 4.2694 - psnr: 3.7925 - ssmi: 0.0064 - sharp_diff: 5.5330 - val_loss: 4.1311 - val_psnr: 3.7633 - val_ssmi: 0.0697 - val_sharp_diff: 11.1242


Epoch 22/50
49/50 [============================>.] - ETA: 13s - loss: 4.2063 - psnr: 3.7230 - ssmi: 0.0060 - sharp_diff: 5.512 - ETA: 13s - loss: 4.1469 - psnr: 3.6630 - ssmi: 0.0056 - sharp_diff: 5.505 - ETA: 12s - loss: 4.1656 - psnr: 3.6811 - ssmi: 0.0056 - sharp_diff: 5.496 - ETA: 12s - loss: 4.1914 - psnr: 3.7067 - ssmi: 0.0057 - sharp_diff: 5.491 - ETA: 11s - loss: 4.3478 - psnr: 3.8634 - ssmi: 0.0070 - sharp_diff: 5.517 - ETA: 11s - loss: 4.3429 - psnr: 3.8582 - ssmi: 0.0071 - sharp_diff: 5.511 - ETA: 11s - loss: 4.3174 - psnr: 3.8326 - ssmi: 0.0067 - sharp_diff: 5.508 - ETA: 10s - loss: 4.3018 - psnr: 3.8169 - ssmi: 0.0063 - sharp_diff: 5.511 - ETA: 10s - loss: 4.3078 - psnr: 3.8227 - ssmi: 0.0065 - sharp_diff: 5.509 - ETA: 10s - loss: 4.3136 - psnr: 3.8283 - ssmi: 0.0065 - sharp_diff: 5.505 - ETA: 9s - loss: 4.3120 - psnr: 3.8264 - ssmi: 0.0064 - sharp_diff: 5.502 - ETA: 9s - loss: 4.2956 - psnr: 3.8099 - ssmi: 0.0063 - sharp_diff: 5.50 - ETA: 9s - loss: 4.2916 - psnr: 3.8057 